## Søk i nasjonalbibliografien 


For å søke i nasjonalbibliografien trengs filen som ligger her på github: 

https://raw.githubusercontent.com/Yoonsen/Modules/master/mods_meta.py

Det greieste er bare å utføre curl-kommandoen nedenfor.

In [3]:
import dhlab.module_update as mu
mu.update("mods_meta")

Updated file `C:\Users\larsj\Documents\GitHub\Miscellaneous-Notebooks\Metadata_nasjonalbibliografien\mods_meta.py`

Deretter er det bare å importere den som for eksempel mm

In [4]:
import mods_meta as mm

Søk er to trinn, først hentes URN-er eller MMSID for treffene, deretter slås de opp med de dataene du ønsker. Her viser jeg hvordan du bruker marc-tuplene for å hente ut data.

Kommandoen er `mm.meta_natbib()` som kan mange parametre, der du også kan legge til krav ved hjelp av marctupler. For eksempel, om du ser etter andre opplag av Hamsun kan du prøve noe sånt, der % er jokertegn og åpne plasser må markeres med mellomrom ' ' :

In [27]:
mm.meta_natbib??

Signature:
mm.meta_natbib(
    children=False,
    yearfrom='',
    yearto='',
    lang=None,
    author=None,
    title=None,
    subtitle=None,
    publisher=None,
    subject=None,
    topic=None,
    marctuples=None,
    limit=20,
)
Source:   
def meta_natbib(children = False, yearfrom = "", yearto = "", lang = None, author = None, title = None, subtitle = None, publisher = None, subject = None, topic = None, marctuples = None, limit=20):
    """ New metadata - search the national bibliography using marc21 tuples """
    params = locals()
    #print(params)
    r = requests.post("https://api.nb.no/ngram/get_urns", json = params)
    return r.json()
File:      c:\users\larsj\documents\github\miscellaneous-notebooks\metadata_nasjonalbibliografien\mods_meta.py
Type:      function


In [25]:
# forfatter er det eget nøkkelord for men for å vise marctuplenes funksjonalitet legger jeg det inn der. 
# Marc for opplag ligger i 250##a, og der ser vi etter tallet 2

korpus = mm.meta_natbib( title = "%evaluering av%", yearfrom = 1980, yearto = 2020)

In [26]:
len(korpus)

5

Variabelen `korpus` inneholder nå en liste av MMSID og URNer (bare serienummeret for digibok-serien) begrenset i størrelse av nøkkelordet `limit`. For å se hva vi har funnet, kjøres kommandoen `mm.metadata()` for de urnene, og med en liste av marctupler vi ønsker verdien på. Her forfatter, tittel og opplag. Kommandoen har noen defaultsettinger for hva som returneres, her har vi fylt inn marc-tupler for navn, tittel og opplag, mens språk og år kommer automatisk. Bare å fylle på!

In [8]:
# i posisjon 2 ligger URNer, første element er mmsid
korpus[:5]

[[991123259474702201, 2015111308007],
 [990717224754702201, 2011090808024],
 [991112247724702201, 2014050805065],
 [999728473324702201, 2011020408139],
 [990602127214702201, 2011071808029]]

Finn metadata for tuplene

In [9]:
meta = mm.metadata(urns = [x[1] for x in korpus], 
                   marctuples=[
                       (100, 1, ' ', 'a'),
                       (245,1,0,'a'),
                       (245,1,0,'b'), 
                       (250, ' ', ' ', 'a'),
                       (260, ' ', ' ', 'a')
                   ]
                  )
meta

{'2011020408139': {'100-1- -a': 'Lorentzen, Hilde',
  '260- - -a': 'Oslo',
  'lang': 'nob',
  'year': 1997},
 '2011071808029': {'100-1- -a': 'Brottveit, Ånund',
  '260- - -a': 'Oslo',
  'lang': 'nob',
  'year': 2006},
 '2011090808024': {'100-1- -a': 'Bjørnstad, Thor Christian',
  '260- - -a': 'Oslo',
  'lang': 'nob',
  'year': 2007},
 '2014050805065': {'100-1- -a': 'Bjørnstad, Roger',
  '260- - -a': 'Oslo',
  'lang': 'nob',
  'year': 2011},
 '2015111308007': {'100-1- -a': 'Negaard, Arve',
  '260- - -a': 'Halden',
  'lang': 'nob',
  'year': 2011}}

# Bruk pandas for å få det hele i tabellform

In [10]:
import pandas as pd

In [11]:
pd.DataFrame(meta).transpose()

,100-1- -a,260- - -a,lang,year
2011020408139,"Lorentzen, Hilde",Oslo,nob,1997
2011071808029,"Brottveit, Ånund",Oslo,nob,2006
2011090808024,"Bjørnstad, Thor Christian",Oslo,nob,2007
2014050805065,"Bjørnstad, Roger",Oslo,nob,2011
2015111308007,"Negaard, Arve",Halden,nob,2011
